https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/centerpartiets-budgetmotion-2022_H9024121

In [1]:
import pandas as pd
import requests

pd.options.mode.chained_assignment = None

In [2]:
multiplier = 1_000_000

In [3]:
search_url = 'https://data.riksdagen.se/dokumentlista/'
params = {
    'sok': 'utgiftsområde',
    'doktyp': 'mot',
    'rm': '2021/22',
    'parti': 'C',
    'sort': 'rel',
    'sortorder': 'desc',
    'utformat': 'json',
    'a': 's'}

resp = requests.get(search_url, params=params)
data = resp.json()
docs = pd.DataFrame(data['dokumentlista']['dokument'])
docs = docs[['titel', 'undertitel', 'rm', 'dokument_url_html']]

In [4]:
docs['Utgiftsområde'] = docs.titel.str.extract('Utgiftsområde (\d+)')
docs['Utgiftsområde'] = docs['Utgiftsområde'].astype(int)
docs = docs.sort_values('Utgiftsområde').reset_index(drop=True)

In [5]:
def find_matching_table(tables, loc=None):
    if loc:
        return tables[loc]
    for table in tables:
        if table.columns.shape == (5,):
            break
    return table

In [56]:
def fetch_table(url, area):
    url = f'http:{url}'
    tables = pd.read_html(url, encoding='utf8', header=2)
    cols = ['Anslag', 'Namn', '2022', '2023', '2024']
    loc = 3 if area in [14, 24] else None
    df = find_matching_table(tables, loc)
    df.columns = cols
    df = df.dropna(how='all')
    df = df[~df.Anslag.str.startswith('Summa', na=False)]
    df['Utgiftsområde'] = area
    return df

In [57]:
tables = []

for _, row in docs.iterrows():
    table = fetch_table(row.dokument_url_html, area=row.Utgiftsområde)
    tables.append(table)

In [58]:
df = pd.concat(tables, sort=False)

In [59]:
df = df.dropna(how='all')

In [61]:
for col in ['2022', '2023', '2024']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.split('.', expand=True)[0]
    df[col] = df[col].str.replace('±0', '0', regex=False)
    df[col] = df[col].str.replace('\s+', '', regex=True)
    df[col] = df[col].str.replace('−', '-')
    df[col] = df[col].astype(int) * multiplier

In [62]:
df.to_csv('../data/budgetmotion-2022-c.csv', index=False)